In [1]:
import numpy as np
from scipy import signal
import math
import abc

from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
from transmux.wola_processor import WolaProcessor
from transmux.utils import gen_real_chirp

In [35]:
# Create WOLA processor
fs = 1.0
num_channels = int(11)

wp = WolaProcessor(fun=None, hop_size=5, block_size=num_channels)
data = 5.6*gen_real_chirp(5000)
data = 100*np.ones_like(data)
samps_per_step = wp.R

plt.rcParams.update({'font.size': 7})

def f(**kwargs):
#     print(kwargs)
    def process_func(x: np.ndarray):
        f = np.fft.fftfreq(x.size, d=1./x.size)
        for c in kwargs:
            x[int(c)] *= 10**(kwargs[c]/20/2)
        return x

    wp.process_func = process_func
    y = np.ndarray((0,), dtype=np.complex128)
    for n in range(0, int((data.size-wp.N)/samps_per_step)):
        inds = np.arange(wp.N) + n*samps_per_step
        d = wp.process(data[inds])
        y = np.hstack((y, d))
        
    print((data[100], y[100]))

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1)
    ax = fig.add_subplot(gs[0, 0])
    f, Pxx_spec = signal.welch(data, fs, 'flattop', 64, return_onesided=False, scaling='spectrum')
#     Pxx_spec /= np.max(Pxx_spec)
    ax.plot(np.fft.fftshift(f), 20*np.log10(np.abs(np.fft.fftshift(Pxx_spec))))
    f, Pxx_spec = signal.welch(y, fs, 'flattop', 64, return_onesided=False, scaling='spectrum')
#     Pxx_spec /= np.max(Pxx_spec)
    ax.plot(np.fft.fftshift(f), 20*np.log10(np.abs(np.fft.fftshift(Pxx_spec))))
    ax.set_xlabel('frequency [Hz]')
    ax.legend(('Original', 'Recombined'), loc='upper right')
    ax.set_title('Output spectrum')
    ax.grid(True)
#     ax.set_ylim(bottom=-15)
#     ax.set_ylim(top=15)

ctl_map = dict()
for n in range(num_channels):
    ctl_map['{}'.format(n)] = widgets.FloatSlider(min=-9,max=9,step=0.5,value=0.0)
    
interact(f, **ctl_map)
#          Pin_dBm=widgets.IntSlider(min=-133,max=-110,step=1,value=-125),
#          BW_Hz=widgets.IntSlider(min=50000,max=500000,step=10000,value=200000));

interactive(children=(FloatSlider(value=0.0, description='0', max=9.0, min=-9.0, step=0.5), FloatSlider(value=…

<function __main__.f(**kwargs)>